In [22]:
import torch
from transformers import AutoModel
from torch.utils.data import DataLoader
from torchvision import transforms
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import transformers
from datasets import load_dataset
import handler

import pathlib
import sklearn
import datasets
import pandas as pd
import glob
import functools
import os
import sys

TOKENIZERS_PARALLELISM=False

In [23]:
!ls ./trainedmodels/


config.json             model_weights.pt        tokenizer.json
model.safetensors       setup_config.json       tokenizer_config.json
model_store             special_tokens_map.json vocab.txt


In [24]:
BASE_DIR = '/Users/tylerklimas/Desktop/BERTModel/TrainedModels'

MODEL_NAME = 'pt_classifier'
MODEL_VERSION = '1.0'
MODEL_STORE='${BASE_DIR}/model_store'
MODEL_SERIALIZED_FILE="${BASE_DIR}/model_weights.pt"

TOKENIZER_FILES= "${BASE_DIR}/tokenizer_config.json, ${BASE_DIR}/special_tokens_map.json, ${BASE_DIR}/vocab.txt,  ${BASE_DIR}/tokenizer.json"
MODEL_EXTRA_FILES="${BASE_DIR}/setup_config.json, ${TOKENIZER_FILES}"


In [26]:
%%bash

BASE_DIR='/Users/tylerklimas/Desktop/BERTModel/TrainedModels'

MODEL_NAME='pt_classifier'
MODEL_VERSION='1.0'
MODEL_STORE="${BASE_DIR}/model_store"
MODEL_SERIALIZED_FILE="${BASE_DIR}/model_weights.pt"

# Ensure no spaces after commas
TOKENIZER_FILES="${BASE_DIR}/tokenizer_config.json,${BASE_DIR}/special_tokens_map.json,${BASE_DIR}/vocab.txt,${BASE_DIR}/tokenizer.json"
MODEL_EXTRA_FILES="${BASE_DIR}/setup_config.json,${TOKENIZER_FILES}"

mkdir -p $MODEL_STORE

torch-model-archiver --model-name ${MODEL_NAME} \
--version ${MODEL_VERSION} \
--serialized-file ${MODEL_SERIALIZED_FILE} \
--export-path ${MODEL_STORE} \
--extra-files "${MODEL_EXTRA_FILES}" \
--handler ./handler.py


In [31]:
!pwd

/Users/tylerklimas/Desktop/BERTModel


In [37]:
%%bash 
torchserve --ts-config /Users/tylerklimas/Desktop/BERTModel/TrainedModels/config.properties \
--start --model-store /Users/tylerklimas/Desktop/BERTModel/TrainedModels/model_store --ncs

In [34]:
# %%bash 
# torchserve --stop

TorchServe has stopped.


In [39]:
!cat /Users/tylerklimas/Desktop/BERTModel/TrainedModels/config.properties

load_models=all
inference_address=http://0.0.0.0:9080
management_address=http://0.0.0.0:9081
metrics_address=http://0.0.0.0:9082
model_store=model_store
async_logging=true

In [42]:
# !curl "http://localhost:9081/models/pt_classifier"

In [ ]:
! curl -X POST http://localhost:9081/predictions/pt_classifier \
        -H 'Content-Type: application/json' \
        -d '{"text": "herbal tea", "request_id":"test_id"}' \
        -w "\nelasped time (sec): %{time_total}"